#### Faiss
Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning.

In [6]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader=TextLoader("speech.txt")
documents=loader.load()
text_splitter=CharacterTextSplitter(chunk_size=1000,chunk_overlap=30)
docs=text_splitter.split_documents(documents)


In [7]:
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\nâ€¦'),
 Document(metadata={'source': 'speech.txt'}, page_content='â€¦\n\nIt will be all the easier for us to conduct

In [21]:
embeddings=OllamaEmbeddings(model="nomic-embed-text")
db=FAISS.from_documents(docs,embeddings)
db

In [12]:
### querying 
query="How does the speaker describe the desired outcome of the war?"
docs=db.similarity_search(query)
docs[0].page_content


'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our heartsâ€”for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

#### As a Retriever
We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers

In [ ]:
retriever=db.as_retriever()
docs=retriever.invoke(query)
docs[0].page_content   

'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our heartsâ€”for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

#### Similarity Search with score
There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better.

In [14]:
docs_and_score=db.similarity_search_with_score(query)
docs_and_score

[(Document(id='2c0c97b6-2986-49ce-859d-1f59a977316f', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our heartsâ€”for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
  343.1831),
 (Document(id='e488dcc0-3622-4f50-b6eb-4acebf8f84c1', metadata={'source': 'speech.txt'}, 

In [22]:
embedding_vector=embeddings.embed_query(query)
embedding_vector

[-0.30195802450180054,
 0.279702365398407,
 -3.61437726020813,
 -0.9989471435546875,
 2.0197343826293945,
 1.4682365655899048,
 -1.0227209329605103,
 0.23442497849464417,
 0.4824120104312897,
 -0.18001538515090942,
 -0.06290391832590103,
 0.5969198942184448,
 0.8668404817581177,
 1.6235871315002441,
 2.0874314308166504,
 -0.7814430594444275,
 0.258811354637146,
 -1.2088690996170044,
 -0.682703971862793,
 0.7261738777160645,
 -0.901149570941925,
 -0.39372920989990234,
 0.092060886323452,
 0.479560524225235,
 1.3447808027267456,
 0.4102768301963806,
 -0.5244477987289429,
 0.6712040901184082,
 -1.0715255737304688,
 -0.5257135629653931,
 0.9153874516487122,
 0.06161807104945183,
 -0.05113380029797554,
 0.28170958161354065,
 -1.5363163948059082,
 -1.9171620607376099,
 0.07945519685745239,
 1.335063099861145,
 0.3556180000305176,
 -1.4278998374938965,
 0.06234705075621605,
 -0.49652189016342163,
 0.12256263941526413,
 -1.5530332326889038,
 1.3662946224212646,
 -0.689235508441925,
 -0.1304230

In [23]:
docs_score=db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(id='2c0c97b6-2986-49ce-859d-1f59a977316f', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our heartsâ€”for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='e488dcc0-3622-4f50-b6eb-4acebf8f84c1', metadata={'source': 'speech.txt'}, page_content='â

In [24]:
### Saving And Loading
db.save_local("faiss_index")

In [25]:
new_db=FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)
docs=new_db.similarity_search(query)

In [26]:
docs

[Document(id='2c0c97b6-2986-49ce-859d-1f59a977316f', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our heartsâ€”for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='e488dcc0-3622-4f50-b6eb-4acebf8f84c1', metadata={'source': 'speech.txt'}, page_content='â